# Installation and configuration

In [2]:
# !pip install -r ./requirements.txt -q

In [3]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

# LangChain

## LLM

In [4]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [5]:
llm = OpenAI(temperature=0.7)
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)


c:\Users\Kevin\Personal\Repositories\Python\Notebooks\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Justin Bieber was born on March 1, 1994. The Super Bowl for the 1993 NFL season was Super Bowl XXVIII, which was played on January 30, 1994. The Dallas Cowboys defeated the Buffalo Bills 30-13 to win their second consecutive Super Bowl. So, the Dallas Cowboys won the Super Bowl in the year Justin Bieber was born.'

## CHAT

In [6]:
from langchain.callbacks import get_openai_callback
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [7]:
chat = ChatOpenAI(temperature=0)
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
with get_openai_callback() as cb:
    print(chat.invoke(messages))
    print(cb)

content="J'adore programmer." response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 34, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None} id='run-886b94d5-acaa-4860-9daf-13f4671cd9cf-0'
Tokens Used: 40
	Prompt Tokens: 34
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $6.300000000000001e-05


## Prompt templates

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
template = """
your task is to translate the following text to {target_language}:

"{text}"
"""

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['target_language', 'text'], template='\nyour task is to translate the following text to {target_language}:\n\n"{text}"\n')

In [10]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo")
chain = LLMChain(prompt=prompt_template, llm=llm)
text = "I love programming a lot in my free time"
response = chain.invoke(input={"text": text, "target_language": "spanish"})
print(response)

{'text': '"Me encanta programar mucho en mi tiempo libre"', 'target_language': 'spanish'}


## Chains

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo")

In [13]:
template1 = """
Create an {amount} of items in a JSON list following the schema below, try to repeat at least 1 product name more than once.:
{{
    name: str,
    amount: int,
}}
"""
prompt_template1 = PromptTemplate.from_template(template1)
chain1 = LLMChain(prompt=prompt_template1, llm=llm)
response = chain1.invoke("4")
print(response["text"])


[
    {
        "name": "Apple",
        "amount": 5
    },
    {
        "name": "Banana",
        "amount": 3
    },
    {
        "name": "Orange",
        "amount": 2
    },
    {
        "name": "Apple",
        "amount": 7
    }
]


In [14]:
template2 = """from the JSON list {data}, get the mean, median, and mode of the data return it in a JSON object."""
prompt_template2 = PromptTemplate.from_template(template2)
chain2 = LLMChain(prompt=prompt_template2, llm=llm)

In [15]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2],verbose=True)
output = overall_chain.invoke("6")



> Entering new SimpleSequentialChain chain...
[
    {
        "name": "apple",
        "amount": 5
    },
    {
        "name": "banana",
        "amount": 3
    },
    {
        "name": "orange",
        "amount": 2
    },
    {
        "name": "apple",
        "amount": 4
    },
    {
        "name": "grapes",
        "amount": 1
    },
    {
        "name": "kiwi",
        "amount": 6
    }
]
{
    "mean": 3.5,
    "median": 3.5,
    "mode": "apple"
}

> Finished chain.


In [16]:
print(output["output"])

{
    "mean": 3.5,
    "median": 3.5,
    "mode": "apple"
}


## ReAct Agent (RAG)
allow your model to interact with real world data or external services, like databases or web APIs.

In [17]:
from langchain.schema import(
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

In [18]:
python_repl = PythonREPL()
python_repl.run('print([n for n in range (1, 10) if n % 2 == 0])')


Python REPL can execute arbitrary code. Use with caution.


'[2, 4, 6, 8]\n'

In [19]:
llm = ChatOpenAI(temperature=0.7 , model_name="gpt-4-turbo-preview")

In [20]:
agent_executor = create_python_agent(llm=llm, tool=PythonREPLTool(),verbose=True)

In [21]:
prompt = 'Find the mean of the first 10 even numbers and print the result, force to display 3 decimals.'
response = agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To find the mean of the first 10 even numbers, I will generate a list of the first 10 even numbers starting from 2, sum them up, and then divide by 10. After that, I'll format the result to display 3 decimal places.
Action: Python_REPL
Action Input: print("{:.3f}".format(sum(range(2, 21, 2)) / 10))
Observation: 11.000

Thought:I now know the final answer.
Final Answer: The mean of the first 10 even numbers is 11.000.

> Finished chain.


In [22]:
response["output"]

'The mean of the first 10 even numbers is 11.000.'

## More advanced RAG

In [26]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

In [50]:
llm = ChatOpenAI(model_name="gpt-4-turbo-preview",temperature=0.7)

In [51]:
template = """
answer the following questions in spanish:
{question}
"""

In [52]:
prompt_template = PromptTemplate.from_template(template)

In [53]:
#the way how langchain decides which agent to use
prompt = hub.pull("hwchase17/react")

### tools available for RAG

In [54]:
#1 python REPL tool
python_repl=PythonREPLTool()
python_repl_tool = Tool(
    name="PythonREPLTool",
    func=python_repl.run,
    description="Util when you need to run python code. You should input python code to use it."
)

#2 wikipedia query tool
api_wrapper = WikipediaAPIWrapper()
wikipedia= WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name="WikipediaTool",
    func=wikipedia.run,
    description="Util when you need to search information inside wikipedia."
)

#3 DuckDuckGo search tool
duckduckgo = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="DuckDuckGoTool",
    func=duckduckgo.run,
    description="Util when you need to search information inside DuckDuckGo. You should use it when you need to find information across the web."
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

In [55]:
agent = create_react_agent(llm=llm, tools=tools,prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [59]:
question = "What is the capital of France?"
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
I know the capital of France is Paris, but I need to translate that into Spanish.

Thought: To translate "Paris" into Spanish, I don't need to use any tool because the name of the city does not change between English and Spanish. However, for the full sentence translation, I should formulate it correctly in Spanish.

Final Answer: La capital de Francia es París.

> Finished chain.
La capital de Francia es París.


In [63]:
question = "What does wikipedia say about France?, first search in english and then translate it into spanish."
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
First, I need to search for information about France on Wikipedia in English.
Action: WikipediaTool
Action Input: FrancePage: France
Summary: France, officially the French Republic, is a country located primarily in Western Europe. It also includes overseas regions and territories in the Americas and the Atlantic, Pacific and Indian oceans, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the north east, Switzerland to the east, Italy and Monaco to the south east, Andorra and Spain to the south, and a maritime border with the United Kingdom to the north west. Its metropolitan area extends from the Rhine to the Atlantic Ocean and from the Mediterranean Sea to the English Channel and the North Sea. Its overseas territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Ind

In [64]:
question = "search how to make a tamal in the web?"
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
To find a recipe for making tamales, I should use the DuckDuckGoTool to search the web for a reliable source.

Action: DuckDuckGoTool
Action Input: how to make a tamal recipePrepare husks and dough: Soak corn husks in a bowl of warm water until softened, about 30 minutes. While the husks are soaking, beat lard with 1 tablespoon broth until fluffy. Combine masa harina, baking powder, and salt in a separate bowl; stir into the lard mixture, adding more broth as needed to form a spongy dough. Drop a quarter-sized ball of masa in a glass filled with cold water. If the masa floats, it is ready. If it sinks, add the remaining 1/4 cup chicken stock, knead for 5 more minutes, and repeat the test. Continue kneading and testing until the ball floats to the top of the glass. Preheat the oven to 400°F and prepare a large baking sheet with aluminum foil. Add the tomatoes, onion and jalapeños and drizzle with oil and season with salt. Toss to coat. Roast for 2

c:\Users\Kevin\Personal\Repositories\Python\Notebooks\.venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


I have found a detailed procedure and ingredients list in English for making tamales. To answer the question in Spanish, I will translate the key steps and ingredients into Spanish.

Final Answer: Para hacer tamales, necesitarás preparar las hojas y la masa. Remoja las hojas de maíz en un tazón con agua tibia hasta que estén suaves, aproximadamente 30 minutos. Mientras las hojas se remojan, bate la manteca con una cucharada de caldo hasta que esté esponjosa. Combina la harina de masa, el polvo de hornear y la sal en un tazón separado; incorpóralos a la mezcla de manteca, agregando más caldo según sea necesario para formar una masa esponjosa. Coloca una bola de masa del tamaño de un cuarto en un vaso lleno de agua fría. Si la masa flota, está lista. Si se hunde, agrega el 1/4 de taza restante de caldo de pollo, amasa durante 5 minutos más, y repite la prueba. Continúa amasando y probando hasta que la bola flote en la parte superior del vaso. Precalienta el horno a 400°F y prepara una ba

In [65]:
question = "what is the square root of 7?"
output= agent_executor.invoke({
    'input': prompt_template.format(question=question)
})
print(output["output"])



> Entering new AgentExecutor chain...
To answer this question in Spanish, I need to calculate the square root of 7 first and then translate the answer into Spanish.

Action: PythonREPLTool
Action Input: import math
math.sqrt(7)Observation: 2.6457513110645907
Thought: Now that I know the square root of 7 is approximately 2.6457513110645907, I can translate this number into Spanish.
Final Answer: La raíz cuadrada de 7 es aproximadamente 2.6457513110645907.

> Finished chain.
La raíz cuadrada de 7 es aproximadamente 2.6457513110645907.
